In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) # Display all columns in a pandas dataframe
import matplotlib.pyplot as plt

from itertools import zip_longest

cwd = Path(os.getcwd())
print(os.listdir("/"))

['bin', 'boot', 'dev', 'etc', 'home', 'lib', 'lib64', 'media', 'mnt', 'opt', 'proc', 'root', 'run', 'sbin', 'srv', 'sys', 'tmp', 'usr', 'var', 'get-docker.sh', 'dxdata-0.36.1-py2.py3-none-any.whl', 'install_r_kernel.R', 'install_r_packages.R', '.dockerenv']


In [2]:
print(os.listdir("/mnt/project/data/processed"))

['anamnesis.csv', 'densitometry.csv', 'fractures.csv', 'hesin_fractures.csv', 'icd10_codes.csv', 'medicaments.csv']


In [3]:
df_anamnesis = pd.read_csv("/mnt/project/data/anamnesis.csv", parse_dates=["p53_i2", "p53_i3"])
df_icd10 = pd.read_csv("/mnt/project/data/processed/icd10_codes.csv", parse_dates=["date_i2"])

name_codings = {
    'eid': 'patientId',
    'p53_i2': 'date_i2',
    'p53_i3': 'date_i3',
    'p21003_i2': 'age_i2',
}

df_anamnesis = df_anamnesis.rename(columns=name_codings).sort_values(by="patientId")
df_anamnesis

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,patientId,date_i2,date_i3,age_i2,p2724_i0,p2724_i1,p2724_i2,p2814_i0,p2814_i1,p2814_i2,p50_i0,p50_i1,p50_i2,p12144_i2,p12144_i3,p21002_i0,p21002_i1,p21002_i2,p21002_i3,p23098_i0,p23098_i1,p23098_i2,p23098_i3,p21001_i0,p21001_i1,p21001_i2,p21001_i3,p23104_i0,p23104_i1,p23104_i2,p23104_i3,p120001,p131848,p131850,p2296_i2,p2443_i0,p2443_i1,p2443_i2,p22130,p20002_i0,p20002_i1,p20002_i2,p3581_i0,p3581_i1,p3581_i2,p130722,p1558_i2,p1568_i2,p1578_i2,p1588_i2,p1598_i2,p1608_i2,p5364_i2,p20116_i2,p26258
11190,1000031,2021-10-21,NaT,79,Yes,NaN,Yes,Yes,NaN,Yes,155.0,NaN,152.0,153.0,NaN,70.4,NaN,73.5,NaN,70.4,NaN,73.5,NaN,29.3028,NaN,31.8127,NaN,29.3,NaN,31.8,NaN,NaN,NaN,NaN,No falls,No,NaN,No,NaN,hypertension,NaN,fracture wrist / colles fracture|whooping coug...,Do not know,NaN,54,NaN,Special occasions only,NaN,NaN,NaN,NaN,NaN,NaN,Never,0.562787
11191,1000344,2015-02-23,NaT,58,No,Not sure - other reason,Yes,No,No,No,153.0,152.0,153.0,154.0,NaN,46.5,49.1,48.3,NaN,46.5,49.1,48.3,NaN,19.8642,21.2517,20.6331,NaN,19.9,21.3,20.6,NaN,No,NaN,NaN,No falls,No,No,No,No,NaN,NaN,NaN,NaN,NaN,56,NaN,One to three times a month,NaN,NaN,NaN,NaN,NaN,NaN,Never,0.094347
0,1000621,2014-10-30,NaT,58,Not sure - had a hysterectomy,Not sure - had a hysterectomy,Not sure - had a hysterectomy,Yes,Yes,Yes,156.0,154.0,156.5,159.0,NaN,54.6,54.4,56.6,NaN,54.6,54.4,56.6,NaN,22.4359,22.9381,23.1094,NaN,22.4,22.9,23.3,NaN,No,NaN,NaN,No falls,No,No,No,No,hypertension|pulmonary embolism +/- dvt|divert...,hypertension|pulmonary embolism +/- dvt|hypoth...,hypertension|pulmonary embolism +/- dvt|hypoth...,NaN,NaN,NaN,NaN,Daily or almost daily,6,28,0,1,0,0,Previous,0.742671
5682,1000898,2018-11-07,NaT,59,No,Yes,Yes,No,Yes,Yes,167.0,167.0,166.0,168.0,NaN,76.1,80.9,85.9,NaN,76.1,80.9,85.9,NaN,27.2867,29.0079,31.1729,NaN,27.3,29.0,31.2,NaN,Do not know,NaN,NaN,More than one fall,No,No,No,No,NaN,fracture toe|depression|prolapsed disc/slipped...,NaN,NaN,48,48,NaN,One to three times a month,NaN,NaN,NaN,NaN,NaN,NaN,Previous,0.258048
16766,1000946,2018-02-12,NaT,67,Yes,NaN,Yes,Yes,NaN,Yes,163.0,NaN,165.0,165.0,NaN,77.9,NaN,84.9,NaN,77.9,NaN,84.9,NaN,29.3199,NaN,31.1846,NaN,29.3,NaN,31.2,NaN,NaN,NaN,NaN,No falls,No,NaN,No,No,NaN,NaN,unclassifiable|osteoarthritis,49,NaN,52,NaN,Once or twice a week,0,3,0,0,0,0,Never,0.907862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28035,6023313,2014-11-20,NaT,72,Yes,NaN,Yes,No,NaN,No,158.0,NaN,157.0,159.0,NaN,81.4,NaN,69.3,NaN,81.4,NaN,69.3,NaN,32.6070,NaN,28.1147,NaN,32.6,NaN,28.1,NaN,No,NaN,NaN,More than one fall,No,NaN,No,No,hypothyroidism/myxoedema|heart valve problem/h...,NaN,hypertension|hypothyroidism/myxoedema|lymphoedema,50,NaN,50,NaN,Never,NaN,NaN,NaN,NaN,NaN,NaN,Never,NaN
11189,6023403,2017-09-26,NaT,75,Yes,NaN,Yes,No,NaN,No,163.5,NaN,162.0,162.0,NaN,84.4,NaN,75.1,NaN,84.4,NaN,75.1,NaN,31.5724,NaN,28.6161,NaN,31.8,NaN,28.6,NaN,NaN,NaN,NaN,No falls,No,NaN,No,No,hypertension|asthma|hypothyroidism/myxoedema|o...,NaN,asthma|hypertension|hayfever/allergic rhinitis...,58,NaN,58,NaN,Never,NaN,NaN,NaN,NaN,NaN,NaN,Never,-1.000480
5681,6023555,2019-06-20,NaT,63,No,NaN,Yes,No,NaN,No,156.0,NaN,155.0,157.0,NaN,63.8,NaN,66.2,NaN,63.8,NaN,66.2,NaN,26.2163,NaN,27.5546,NaN,26.2,NaN,27.6,NaN,NaN,NaN,NaN,Only one fall,No,NaN,No,No,thyroid problem (not cancer)|allergy or anaphy...,NaN,asthma|glaucoma|eczema/dermatitis|thyroid goit...,NaN,NaN,57,NaN,One to three times a month,NaN,NaN,NaN,NaN,NaN,NaN,Never,-0.322836
16764,6023580,2019-10-25,NaT,57,No,NaN,Yes,No,NaN,No,164.0,NaN,164.0,165.0,NaN,66.8,NaN,64.4,NaN,66.8,NaN,64.4,NaN,24.8364,NaN,23.9441,NaN,24.8,NaN,23.9,NaN,No,NaN,NaN,No falls,No,NaN,No,No,depression,NaN,NaN,NaN,NaN,Do not know,NaN,Three or four times a week,4,0,0,2,0,0,Never,-0.014252


In [4]:
def forward_fill_variable(row, fields, field_type):
    if field_type == "yes_no":
        for field in fields[::-1]:
            if row[field] == "Yes" or row[field] == True:
                return True
        return False
    elif field_type == "menopause_u45":
        for field in fields[::-1]:
            try:
                age = int(row[field])
            except ValueError:
                continue
            if age < 45:
                return True
            else:
                return False
        return False  # All values are NaN -> set to False
    raise ValueError(f"field_type '{field_type}' not supported.")

In [5]:
# Forward fill menopause status and store value in a new column
menopausal_variables = ["p2724_i0", "p2724_i1", "p2724_i2"]
df_anamnesis["postmenopausal"] = df_anamnesis[menopausal_variables].apply(lambda x: forward_fill_variable(x, menopausal_variables, "yes_no"), axis=1)

# Drop old columns
df_anamnesis.drop(menopausal_variables, axis=1, inplace=True)

# Filter for menopausal women
pm_women = df_anamnesis[df_anamnesis["postmenopausal"] == True].reset_index(drop=True)
pm_women

,patientId,date_i2,date_i3,age_i2,p2814_i0,p2814_i1,p2814_i2,p50_i0,p50_i1,p50_i2,p12144_i2,p12144_i3,p21002_i0,p21002_i1,p21002_i2,p21002_i3,p23098_i0,p23098_i1,p23098_i2,p23098_i3,p21001_i0,p21001_i1,p21001_i2,p21001_i3,p23104_i0,p23104_i1,p23104_i2,p23104_i3,p120001,p131848,p131850,p2296_i2,p2443_i0,p2443_i1,p2443_i2,p22130,p20002_i0,p20002_i1,p20002_i2,p3581_i0,p3581_i1,p3581_i2,p130722,p1558_i2,p1568_i2,p1578_i2,p1588_i2,p1598_i2,p1608_i2,p5364_i2,p20116_i2,p26258,postmenopausal
0,1000031,2021-10-21,NaT,79,Yes,NaN,Yes,155.0,NaN,152.0,153.0,NaN,70.4,NaN,73.5,NaN,70.4,NaN,73.5,NaN,29.3028,NaN,31.8127,NaN,29.3,NaN,31.8,NaN,NaN,NaN,NaN,No falls,No,NaN,No,NaN,hypertension,NaN,fracture wrist / colles fracture|whooping coug...,Do not know,NaN,54,NaN,Special occasions only,NaN,NaN,NaN,NaN,NaN,NaN,Never,0.562787,True
1,1000344,2015-02-23,NaT,58,No,No,No,153.0,152.0,153.0,154.0,NaN,46.5,49.1,48.3,NaN,46.5,49.1,48.3,NaN,19.8642,21.2517,20.6331,NaN,19.9,21.3,20.6,NaN,No,NaN,NaN,No falls,No,No,No,No,NaN,NaN,NaN,NaN,NaN,56,NaN,One to three times a month,NaN,NaN,NaN,NaN,NaN,NaN,Never,0.094347,True
2,1000898,2018-11-07,NaT,59,No,Yes,Yes,167.0,167.0,166.0,168.0,NaN,76.1,80.9,85.9,NaN,76.1,80.9,85.9,NaN,27.2867,29.0079,31.1729,NaN,27.3,29.0,31.2,NaN,Do not know,NaN,NaN,More than one fall,No,No,No,No,NaN,fracture toe|depression|prolapsed disc/slipped...,NaN,NaN,48,48,NaN,One to three times a month,NaN,NaN,NaN,NaN,NaN,NaN,Previous,0.258048,True
3,1000946,2018-02-12,NaT,67,Yes,NaN,Yes,163.0,NaN,165.0,165.0,NaN,77.9,NaN,84.9,NaN,77.9,NaN,84.9,NaN,29.3199,NaN,31.1846,NaN,29.3,NaN,31.2,NaN,NaN,NaN,NaN,No falls,No,NaN,No,No,NaN,NaN,unclassifiable|osteoarthritis,49,NaN,52,NaN,Once or twice a week,0,3,0,0,0,0,Never,0.907862,True
4,1001273,2019-09-14,NaT,54,No,NaN,No,164.0,NaN,165.0,164.0,NaN,84.0,NaN,91.0,NaN,84.0,NaN,91.0,NaN,31.2314,NaN,33.4252,NaN,31.2,NaN,33.4,NaN,No,NaN,NaN,No falls,No,NaN,No,No,cataract,NaN,NaN,NaN,NaN,52,NaN,Special occasions only,NaN,NaN,NaN,NaN,NaN,NaN,Never,-0.309661,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23532,6023313,2014-11-20,NaT,72,No,NaN,No,158.0,NaN,157.0,159.0,NaN,81.4,NaN,69.3,NaN,81.4,NaN,69.3,NaN,32.6070,NaN,28.1147,NaN,32.6,NaN,28.1,NaN,No,NaN,NaN,More than one fall,No,NaN,No,No,hypothyroidism/myxoedema|heart valve problem/h...,NaN,hypertension|hypothyroidism/myxoedema|lymphoedema,50,NaN,50,NaN,Never,NaN,NaN,NaN,NaN,NaN,NaN,Never,NaN,True
23533,6023403,2017-09-26,NaT,75,No,NaN,No,163.5,NaN,162.0,162.0,NaN,84.4,NaN,75.1,NaN,84.4,NaN,75.1,NaN,31.5724,NaN,28.6161,NaN,31.8,NaN,28.6,NaN,NaN,NaN,NaN,No falls,No,NaN,No,No,hypertension|asthma|hypothyroidism/myxoedema|o...,NaN,asthma|hypertension|hayfever/allergic rhinitis...,58,NaN,58,NaN,Never,NaN,NaN,NaN,NaN,NaN,NaN,Never,-1.000480,True
23534,6023555,2019-06-20,NaT,63,No,NaN,No,156.0,NaN,155.0,157.0,NaN,63.8,NaN,66.2,NaN,63.8,NaN,66.2,NaN,26.2163,NaN,27.5546,NaN,26.2,NaN,27.6,NaN,NaN,NaN,NaN,Only one fall,No,NaN,No,No,thyroid problem (not cancer)|allergy or anaphy...,NaN,asthma|glaucoma|eczema/dermatitis|thyroid goit...,NaN,NaN,57,NaN,One to three times a month,NaN,NaN,NaN,NaN,NaN,NaN,Never,-0.322836,True
23535,6023580,2019-10-25,NaT,57,No,NaN,No,164.0,NaN,164.0,165.0,NaN,66.8,NaN,64.4,NaN,66.8,NaN,64.4,NaN,24.8364,NaN,23.9441,NaN,24.8,NaN,23.9,NaN,No,NaN,NaN,No falls,No,NaN,No,No,depression,NaN,NaN,NaN,NaN,Do not know,NaN,Three or four times a week,4,0,0,2,0,0,Never,-0.014252,True


In [6]:
# forward fill HRT variable
hrt_variables = ["p2814_i0", "p2814_i1", "p2814_i2"]
pm_women["hrt"] = pm_women[hrt_variables].apply(lambda x: forward_fill_variable(x, hrt_variables, "yes_no"), axis=1)
pm_women.drop(hrt_variables, axis=1, inplace=True)

### Prepare height, weight and BMI variables

In [7]:
body_measurements = pm_women.loc[:,"p50_i0":"p23104_i3"]
body_measurement_columns = body_measurements.columns.tolist()
body_measurements

,p50_i0,p50_i1,p50_i2,p12144_i2,p12144_i3,p21002_i0,p21002_i1,p21002_i2,p21002_i3,p23098_i0,p23098_i1,p23098_i2,p23098_i3,p21001_i0,p21001_i1,p21001_i2,p21001_i3,p23104_i0,p23104_i1,p23104_i2,p23104_i3
0,155.0,NaN,152.0,153.0,NaN,70.4,NaN,73.5,NaN,70.4,NaN,73.5,NaN,29.3028,NaN,31.8127,NaN,29.3,NaN,31.8,NaN
1,153.0,152.0,153.0,154.0,NaN,46.5,49.1,48.3,NaN,46.5,49.1,48.3,NaN,19.8642,21.2517,20.6331,NaN,19.9,21.3,20.6,NaN
2,167.0,167.0,166.0,168.0,NaN,76.1,80.9,85.9,NaN,76.1,80.9,85.9,NaN,27.2867,29.0079,31.1729,NaN,27.3,29.0,31.2,NaN
3,163.0,NaN,165.0,165.0,NaN,77.9,NaN,84.9,NaN,77.9,NaN,84.9,NaN,29.3199,NaN,31.1846,NaN,29.3,NaN,31.2,NaN
4,164.0,NaN,165.0,164.0,NaN,84.0,NaN,91.0,NaN,84.0,NaN,91.0,NaN,31.2314,NaN,33.4252,NaN,31.2,NaN,33.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23532,158.0,NaN,157.0,159.0,NaN,81.4,NaN,69.3,NaN,81.4,NaN,69.3,NaN,32.6070,NaN,28.1147,NaN,32.6,NaN,28.1,NaN
23533,163.5,NaN,162.0,162.0,NaN,84.4,NaN,75.1,NaN,84.4,NaN,75.1,NaN,31.5724,NaN,28.6161,NaN,31.8,NaN,28.6,NaN
23534,156.0,NaN,155.0,157.0,NaN,63.8,NaN,66.2,NaN,63.8,NaN,66.2,NaN,26.2163,NaN,27.5546,NaN,26.2,NaN,27.6,NaN
23535,164.0,NaN,164.0,165.0,NaN,66.8,NaN,64.4,NaN,66.8,NaN,64.4,NaN,24.8364,NaN,23.9441,NaN,24.8,NaN,23.9,NaN


In [8]:
body_measurements.isna().sum()

p50_i0          22
p50_i1       19961
p50_i2         718
p12144_i2        5
p12144_i3    21366
p21002_i0       33
p21002_i1    19960
p21002_i2      740
p21002_i3    21382
p23098_i0      288
p23098_i1    19978
p23098_i2     1253
p23098_i3    21428
p21001_i0       36
p21001_i1    19961
p21001_i2      863
p21001_i3    21382
p23104_i0      288
p23104_i1    19978
p23104_i2     1253
p23104_i3    21428
dtype: int64

In [9]:
# Find number on NaN's for height, weight and BMI variables
height_column = "p12144_i2"  # height (instance 2)
weight_column = "p21002_i2"  # weight (instance 2)
bmi_column = "p21001_i2"     # bmi (instance 2)
relevant_columns = [height_column, weight_column, bmi_column]
body_measurements[relevant_columns].isna().sum()

p12144_i2      5
p21002_i2    740
p21001_i2    863
dtype: int64

In [10]:
# Impute height variable by using another height measurement (manually):
body_measurements[body_measurements[height_column].isna()]

,p50_i0,p50_i1,p50_i2,p12144_i2,p12144_i3,p21002_i0,p21002_i1,p21002_i2,p21002_i3,p23098_i0,p23098_i1,p23098_i2,p23098_i3,p21001_i0,p21001_i1,p21001_i2,p21001_i3,p23104_i0,p23104_i1,p23104_i2,p23104_i3
6573,161.0,NaN,161.0,NaN,NaN,76.0,NaN,78.9,NaN,76.0,NaN,78.9,NaN,29.3199,NaN,30.4386,NaN,29.3,NaN,30.4,NaN
7681,154.0,NaN,150.0,NaN,NaN,63.6,NaN,63.7,NaN,63.6,NaN,63.7,NaN,26.8173,NaN,28.3111,NaN,26.8,NaN,28.3,NaN
10160,167.0,166.0,166.0,NaN,NaN,60.2,63.3,63.8,NaN,60.2,63.3,63.8,NaN,21.5856,22.9714,23.1529,NaN,21.6,23.0,23.2,NaN
18505,169.0,NaN,167.0,NaN,NaN,80.7,NaN,90.8,NaN,80.7,NaN,90.8,NaN,28.2553,NaN,32.5576,NaN,28.3,NaN,32.6,NaN
23482,164.0,NaN,NaN,NaN,NaN,83.4,NaN,NaN,NaN,83.4,NaN,NaN,NaN,31.0083,NaN,NaN,NaN,31.0,NaN,NaN,NaN


In [11]:
body_measurements.loc[6573, "p12144_i2"] = 161.0
body_measurements.loc[7681, "p12144_i2"] = 150.0
body_measurements.loc[10160, "p12144_i2"] = 166.0
body_measurements.loc[18505, "p12144_i2"] = 167.0
body_measurements.loc[23482, "p12144_i2"] = 164.0

In [12]:
# Impute weight variable by using another weight measurement from the following order.
# We have two different weight measurements per instance: "body size measurements" (denoted as weight_bsm) 
# and "body composition by impedance" (denoted as weight_bci)
# 1. p23098_i2: weight_bci (instance 2)
# 2. p21002_i1: weight_bsm (instance 1)
# 3. p23098_i1: weight_bci (instance 1)
# 4. p21002_i0: weight_bsm (instance 0)
# 5. p23098_i0: weight_bci (instance 0)
# 6. p21002_i3: weight_bsm (instance 3)
# 7. p23098_i3: weight_bci (instance 3)

weight_variables = ["p23098_i2", "p21002_i1", "p23098_i1", "p21002_i0", "p23098_i0", "p21002_i3", "p23098_i3"]
for idx, row in body_measurements[body_measurements[weight_column].isna()].iterrows():
    for other_weight_col in weight_variables:
        if pd.notnull(row[other_weight_col]):
            body_measurements.loc[idx, weight_column] = row[other_weight_col]
            break

# Fill remaining NaN's with populations mean
weight_mean = body_measurements.loc[:, weight_column].mean()
body_measurements.loc[:, weight_column].fillna(weight_mean, inplace=True)

# Check for remaining NaN's
body_measurements[relevant_columns].isna().sum()

p12144_i2      0
p21002_i2      0
p21001_i2    863
dtype: int64

In [13]:
# Fill BMI value by using formula: bmi = weight / (height^2)
body_measurements[bmi_column] = round(body_measurements[relevant_columns].apply(lambda x: x[weight_column] / (x[height_column]**2) if pd.isnull(x[bmi_column]) else x[bmi_column], axis=1), 4)
# Check for remaining NaN's
body_measurements[relevant_columns].isna().sum()

p12144_i2    0
p21002_i2    0
p21001_i2    0
dtype: int64

In [14]:
body_measurements.rename(columns={height_column: "height", weight_column: "weight", bmi_column: "bmi"}, inplace=True)
pm_women = pd.concat([pm_women, body_measurements[["height", "weight", "bmi"]]], axis=1)

# Find patients with decrease in height
pm_women["decrease in height"] = (pm_women["p50_i0"] - pm_women["height"] > 2.5).fillna(0)

pm_women.drop(body_measurement_columns, axis=1, inplace=True)
pm_women

,patientId,date_i2,date_i3,age_i2,p120001,p131848,p131850,p2296_i2,p2443_i0,p2443_i1,p2443_i2,p22130,p20002_i0,p20002_i1,p20002_i2,p3581_i0,p3581_i1,p3581_i2,p130722,p1558_i2,p1568_i2,p1578_i2,p1588_i2,p1598_i2,p1608_i2,p5364_i2,p20116_i2,p26258,postmenopausal,hrt,height,weight,bmi,decrease in height
0,1000031,2021-10-21,NaT,79,NaN,NaN,NaN,No falls,No,NaN,No,NaN,hypertension,NaN,fracture wrist / colles fracture|whooping coug...,Do not know,NaN,54,NaN,Special occasions only,NaN,NaN,NaN,NaN,NaN,NaN,Never,0.562787,True,True,153.0,73.5,31.8127,False
1,1000344,2015-02-23,NaT,58,No,NaN,NaN,No falls,No,No,No,No,NaN,NaN,NaN,NaN,NaN,56,NaN,One to three times a month,NaN,NaN,NaN,NaN,NaN,NaN,Never,0.094347,True,False,154.0,48.3,20.6331,False
2,1000898,2018-11-07,NaT,59,Do not know,NaN,NaN,More than one fall,No,No,No,No,NaN,fracture toe|depression|prolapsed disc/slipped...,NaN,NaN,48,48,NaN,One to three times a month,NaN,NaN,NaN,NaN,NaN,NaN,Previous,0.258048,True,True,168.0,85.9,31.1729,False
3,1000946,2018-02-12,NaT,67,NaN,NaN,NaN,No falls,No,NaN,No,No,NaN,NaN,unclassifiable|osteoarthritis,49,NaN,52,NaN,Once or twice a week,0,3,0,0,0,0,Never,0.907862,True,True,165.0,84.9,31.1846,False
4,1001273,2019-09-14,NaT,54,No,NaN,NaN,No falls,No,NaN,No,No,cataract,NaN,NaN,NaN,NaN,52,NaN,Special occasions only,NaN,NaN,NaN,NaN,NaN,NaN,Never,-0.309661,True,False,164.0,91.0,33.4252,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23532,6023313,2014-11-20,NaT,72,No,NaN,NaN,More than one fall,No,NaN,No,No,hypothyroidism/myxoedema|heart valve problem/h...,NaN,hypertension|hypothyroidism/myxoedema|lymphoedema,50,NaN,50,NaN,Never,NaN,NaN,NaN,NaN,NaN,NaN,Never,NaN,True,False,159.0,69.3,28.1147,False
23533,6023403,2017-09-26,NaT,75,NaN,NaN,NaN,No falls,No,NaN,No,No,hypertension|asthma|hypothyroidism/myxoedema|o...,NaN,asthma|hypertension|hayfever/allergic rhinitis...,58,NaN,58,NaN,Never,NaN,NaN,NaN,NaN,NaN,NaN,Never,-1.000480,True,False,162.0,75.1,28.6161,False
23534,6023555,2019-06-20,NaT,63,NaN,NaN,NaN,Only one fall,No,NaN,No,No,thyroid problem (not cancer)|allergy or anaphy...,NaN,asthma|glaucoma|eczema/dermatitis|thyroid goit...,NaN,NaN,57,NaN,One to three times a month,NaN,NaN,NaN,NaN,NaN,NaN,Never,-0.322836,True,False,157.0,66.2,27.5546,False
23535,6023580,2019-10-25,NaT,57,No,NaN,NaN,No falls,No,NaN,No,No,depression,NaN,NaN,NaN,NaN,Do not know,NaN,Three or four times a week,4,0,0,2,0,0,Never,-0.014252,True,False,165.0,64.4,23.9441,False


In [15]:
# Use PRS for osteoporosis as a proxy for osteoporotic fracture parents
# Since 5% of participants have suffered a osteoporotic fracture in our dataset
# we are going to take the top 5 percentile and asssume that they suffered a 
# osteoporotic fracture

threshold = pm_women["p26258"].quantile(0.95)
pm_women["osteoporotic_fracture_parents"] = (pm_women["p26258"] > threshold)
pm_women.drop("p26258", axis=1, inplace=True)
pm_women["osteoporotic_fracture_parents"].sum()

1140

### Prepare variables from ICD-10 codes

These are:
* corticosteroids
* aromatase inhibitors
* anti-epileptic drugs
* morbus bechterew
* diabetes
* copd
* hyperpara

In [16]:
def find_icd10_code(df, codes):
    # Define date and value columns
    values = df.loc[:,"p41270_a0":"p41270_a118"]
    dates = df.loc[:,"p41280_a0":"p41280_a118"]

    # We want to extract only the dates where the code has been found
    values = pd.concat(
        [
            df[value_column].str.startswith(tuple(codes))
            for value_column in values.columns
        ],
        axis=1
    )
    dates = pd.concat(
        [
            pd.to_datetime(df[date_column]).where(values[value_column], pd.NaT)
            for value_column, date_column in zip_longest(values.columns, dates.columns)
        ],
        axis=1
    )

    # Find the first event
    events = (dates.min(axis=1) < df["date_i2"])

    return events

In [17]:
pm_women["corticosteroids"] = find_icd10_code(df_icd10, ["Y42.0"])
pm_women["corticosteroids"].sum()

2

In [18]:
pm_women["aromatase_inhibitors"] = find_icd10_code(df_icd10, ["Y42.6"])
pm_women["aromatase_inhibitors"].sum()

5

In [19]:
pm_women["antiepileptic_drugs"] = find_icd10_code(df_icd10, ["Y46.0", "Y46.1", "Y46.2", "Y46.3", "Y46.4", "Y46.5", "Y46.6"])
pm_women["antiepileptic_drugs"].sum()

3

In [20]:
pm_women["ankylosing_spondylitis"] = find_icd10_code(df_icd10, ["M45", "M46", "M47", "M48", "M49"])
pm_women["ankylosing_spondylitis"].sum()

562

In [21]:
pm_women["copd"] = find_icd10_code(df_icd10, ["J44"])
pm_women.drop("p22130", axis=1, inplace=True)  # not needed copd variable (follow-up questionaire)
pm_women["copd"].sum()

191

In [22]:
pm_women["hyperpara"] = find_icd10_code(df_icd10, ["E21"])
pm_women.drop("p130722", axis=1, inplace=True)  # not needed hyperpara variable
pm_women["hyperpara"].sum()

53

### Prepare variables from ICD10 codes as well as other data fields

In [23]:
# from icd-10 code fields
pm_women["type_1_diabetes"] = find_icd10_code(df_icd10, ["E10", "E11", "E12", "E13", "E14"])

# from touchscreen fields
diabetes_variables = ["p2443_i0", "p2443_i1", "p2443_i2"]
diabetes_patients = pm_women[diabetes_variables].apply(lambda x: forward_fill_variable(x, diabetes_variables, "yes_no"), axis=1)

# Combine by logistic OR
pm_women["type_1_diabetes"] = pm_women["type_1_diabetes"] | diabetes_patients
pm_women.drop(diabetes_variables, axis=1, inplace=True)

pm_women["type_1_diabetes"].sum()

1005

In [24]:
# from icd-10 code fields
pm_women["rheumatoid_arthritis"] = find_icd10_code(df_icd10, ["M05", "M06"])

# from online_follow_up fields
ra_patients = pm_women["p120001"].apply(lambda x: True if x == "Yes" else False)

# Combine by logistic OR
pm_women["rheumatoid_arthritis"] = pm_women["rheumatoid_arthritis"] | ra_patients
pm_women.drop(["p120001", "p131848", "p131850"], axis=1, inplace=True)

pm_women["rheumatoid_arthritis"].sum()

938

In [25]:
pm_women["number_of_falls"] = pm_women["p2296_i2"].where(pm_women["p2296_i2"] != "Prefer not to answer", "No falls")
pm_women["number_of_falls"] = pm_women["number_of_falls"].fillna("No falls")
pm_women.drop("p2296_i2", axis=1, inplace=True)

pm_women["number_of_falls"].value_counts()

No falls              18287
Only one fall          3831
More than one fall     1419
Name: number_of_falls, dtype: int64

In [26]:
menopausal_age_variables = ["p3581_i0", "p3581_i1", "p3581_i2"]
pm_women["early_menopause"] = pm_women[menopausal_age_variables].apply(lambda x: forward_fill_variable(x, menopausal_age_variables, "menopause_u45"), axis=1)
pm_women.drop(menopausal_age_variables, axis=1, inplace=True)
pm_women["early_menopause"].sum()

2464

In [27]:
pm_women["nicotin"] = pm_women["p20116_i2"].apply(lambda x: True if x == "Current" else False)
pm_women.drop("p20116_i2", axis=1, inplace=True)

pm_women["nicotin"].sum()

644

In [28]:
def _get_alcohol_units(patient):
    units = patient[["p1568_i2", "p1578_i2", "p1588_i2", "p1598_i2", "p1608_i2", "p5364_i2"]]
    units = units.apply(lambda x: int(x) if x.isdigit() else -100)
    return units.sum()

def check_alcohol_status(row):
    if row["p1558_i2"] == "Daily or almost daily":
        alcohol_units = _get_alcohol_units(row)
        if alcohol_units >= 21:
            return True
        elif alcohol_units < 0:
            # This is the case when somebody answered "Do not know" or
            # "Prefer not to answer" for one category. 
            # These cases should be further investigated
            return None
    return False

In [29]:
pm_women["alcohol"] = pm_women.apply(lambda x: check_alcohol_status(x), axis=1)
print("Patients that answered 'Do not know' or 'prefer not to answer':", pm_women["alcohol"].isna().sum())
print(pm_women["alcohol"].sum())

Patients that answered 'Do not know' or 'prefer not to answer': 40
577


In [30]:
# Further investigate these patients
alcohol_variables = ["p1558_i2", "p1568_i2", "p1578_i2", "p1588_i2", "p1598_i2", "p1608_i2", "p5364_i2"]
pm_women[alcohol_variables][pm_women["alcohol"].isna()]

,p1558_i2,p1568_i2,p1578_i2,p1588_i2,p1598_i2,p1608_i2,p5364_i2
86,Daily or almost daily,2,7,0,Do not know,0,0
2442,Daily or almost daily,5,Do not know,Prefer not to answer,7,Do not know,Do not know
2742,Daily or almost daily,0,0,8,Do not know,0,0
2882,Daily or almost daily,0,0,0,Prefer not to answer,0,0
3691,Daily or almost daily,Prefer not to answer,0,0,0,0,0
4472,Daily or almost daily,0,Prefer not to answer,0,0,0,0
5750,Daily or almost daily,Prefer not to answer,0,0,0,0,0
9175,Daily or almost daily,Do not know,8,0,7,1,0
9209,Daily or almost daily,Prefer not to answer,14,Do not know,Do not know,Do not know,Do not know
9654,Daily or almost daily,Do not know,Do not know,0,0,0,0


In [31]:
# Set alcohol usage of all patients with more or equal than 21 units to True
for idx, row in pm_women[alcohol_variables][pm_women["alcohol"].isna()].iterrows():
    units = row.loc["p1568_i2":].apply(lambda x: int(x) if x.isdigit() else 0).sum()
    if units >= 21:
        pm_women.loc[idx, "alcohol"] = True

In [32]:
pm_women.drop(alcohol_variables, axis=1, inplace=True)

In [33]:
# investigate inflammatory bowel disease from non-cancer illness codes
ibd_columns = ["p20002_i0", "p20002_i1", "p20002_i2"]

df_ibd = pd.concat(
    [
        pm_women[col].str.contains(r'\b(?:{})\b'.format('|'.join(["inflammatory bowel disease", "crohns disease", "ulcerative colitis"])))
        for col in ibd_columns
    ], 
    axis=1,
)
df_ibd.sum()

p20002_i0    193.0
p20002_i1     35.0
p20002_i2    203.0
dtype: float64

In [34]:
pm_women["gastrointestinal_disease"] = df_ibd.fillna(False).apply(lambda x: forward_fill_variable(x, ibd_columns, "yes_no"), axis=1)
pm_women.drop(ibd_columns, axis=1, inplace=True)

pm_women["gastrointestinal_disease"].sum()

276

In [35]:
pm_women.drop("date_i3", axis=1, inplace=True)
print(pm_women.isna().sum())
pm_women

patientId                         0
date_i2                           0
age_i2                            0
postmenopausal                    0
hrt                               0
height                            0
weight                            0
bmi                               0
decrease in height                0
osteoporotic_fracture_parents     0
corticosteroids                   0
aromatase_inhibitors              0
antiepileptic_drugs               0
ankylosing_spondylitis            0
copd                              0
hyperpara                         0
type_1_diabetes                   0
rheumatoid_arthritis              0
number_of_falls                   0
early_menopause                   0
nicotin                           0
alcohol                          36
gastrointestinal_disease          0
dtype: int64


,patientId,date_i2,age_i2,postmenopausal,hrt,height,weight,bmi,decrease in height,osteoporotic_fracture_parents,corticosteroids,aromatase_inhibitors,antiepileptic_drugs,ankylosing_spondylitis,copd,hyperpara,type_1_diabetes,rheumatoid_arthritis,number_of_falls,early_menopause,nicotin,alcohol,gastrointestinal_disease
0,1000031,2021-10-21,79,True,True,153.0,73.5,31.8127,False,False,False,False,False,False,False,False,False,False,No falls,False,False,False,False
1,1000344,2015-02-23,58,True,False,154.0,48.3,20.6331,False,False,False,False,False,False,False,False,False,False,No falls,False,False,False,False
2,1000898,2018-11-07,59,True,True,168.0,85.9,31.1729,False,False,False,False,False,False,False,False,False,False,More than one fall,False,False,False,False
3,1000946,2018-02-12,67,True,True,165.0,84.9,31.1846,False,False,False,False,False,False,False,False,False,False,No falls,False,False,False,False
4,1001273,2019-09-14,54,True,False,164.0,91.0,33.4252,False,False,False,False,False,False,False,False,False,False,No falls,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23532,6023313,2014-11-20,72,True,False,159.0,69.3,28.1147,False,False,False,False,False,False,False,False,False,False,More than one fall,False,False,False,False
23533,6023403,2017-09-26,75,True,False,162.0,75.1,28.6161,False,False,False,False,False,False,False,False,False,False,No falls,False,False,False,False
23534,6023555,2019-06-20,63,True,False,157.0,66.2,27.5546,False,False,False,False,False,False,False,False,False,False,Only one fall,False,False,False,False
23535,6023580,2019-10-25,57,True,False,165.0,64.4,23.9441,False,False,False,False,False,False,False,False,False,False,No falls,False,False,False,False


**Save and upload to project storage**

In [36]:
pm_women.to_csv("anamnesis.csv", index=False)

In [37]:
%%bash
dx upload anamnesis.csv --path /data/processed/

ID                          file-GPgzXQ8Jjxx8Z4Gpq2KbYKv6
Class                       file
Project                     project-GP77K38Jjxx9XzFP2KzPQyfG
Folder                      /data/processed
Name                        anamnesis.csv
State                       closing
Visibility                  visible
Types                       -
Properties                  -
Tags                        -
Outgoing links              -
Created                     Fri Feb 17 22:01:37 2023
Created by                  ollehman
 via the job                job-GPgz8g8JjxxKxb2XZzg5KPq1
Last modified               Fri Feb 17 22:01:39 2023
Media type                  
archivalState               "live"
cloudAccount                "cloudaccount-dnanexus"
